In [3]:
import pandas as pd
import math
import os

In [4]:
class Subject:
    def __init__(self, name, descriptions):
        self.descriptions = descriptions
        self.name = name
        
class Description:
    def __init__(self, subject_name, university, description):
        self.name = subject_name
        self.university = university
        self.description = description
    
    def __to_dict__(self):
        return {
            'Subject': self.name,
            'University': self.university,
            'Description': self.description
        }
    
class V2L_Data:
    def __init__(self, paper, university, university_subject_name, topics, CO, CO_time, branch=None):
        self.paper = paper
        self.university = university
        self.university_subject_name = university_subject_name
        self.topics = int(topics) if not pd.isna(topics) else topics
        self.CO = int(CO) if not pd.isna(CO) else CO
        self.CO_time = int(CO_time) if not pd.isna(CO_time) else CO_time
        self.branch = branch

In [5]:
def get_subjects(df, is_diploma=False):
    a = df.loc[df['S.N'].notnull()]
    subj_name = 'Subject Name' if not is_diploma else 'Subject Name_ES'
    
    subjects = {}
    subject_cutoffs = []
    for idx, row in a.iterrows():
        subject_cutoffs.append(idx)
    for i in subject_cutoffs:
        subjects[i] = df.iloc[i][subj_name]
    
    return subject_cutoffs,subjects

def extract_info(df, is_diploma=False):
    indices,subjects = get_subjects(df, is_diploma)
    
    if is_diploma:
        subj_name, univ_name, univ_subj_name = 'Subject Name_ES', 'BTE short name', 'Subject NameBTE'
    else:
        subj_name, univ_name, univ_subj_name = 'Subject Name', 'University short name', 'University Subject Name'
    
    result = {}
    pws = {}
    for i,subject in enumerate(subjects.keys()):
        name = subjects[subject].split("(")[0].strip()
        password = df[df[subj_name]==subjects[subject]].Password.item()
        if i == len(subjects)-1:
            val = df[indices[i]:]

        else:
            val = df[indices[i]:indices[i+1]]
        
        data_rows = {}
        for value,g in val.groupby([univ_name]):
            university = value
            data = []
            prev = None
            for x, row in enumerate(g.iterrows()):
                row = row[1]
                    
                if prev != None:
                    if prev.topics == row['# Topics'] and prev.CO == row['# CLOs']:
                        # Same paper code as before
                        paper = prev.paper + "/" + row['Paper code']
                        data[-1].paper = paper
                        continue

                print(name, university,row['# CLOs'])
                d = V2L_Data(row['Paper code'], university, row[univ_subj_name], row['# Topics'], row['# CLOs'], row['CLO time'])            
                prev = d
                data.append(d)
            
            data_rows[university] = data
        
        
        result[name] = {'data':data_rows, 'password': password}
    
    return result

In [21]:
def get_tabby_data_v2l(university_data):
    final_tabby = ''
    #draft = '[tabby title={0}]\n\n{1} offers this course for <strong><u>{2}</u></strong> branches.\n<ul><li><strong>{3} Video Learning Outcomes</strong> attained across<strong> {4} topics</strong> taught over <strong>{5} hours</strong> of instruction.</li><li>Total duration of course: <strong>{5} hours</strong></li></ul>'
    draft = '[tabby title={0}]\n\n{1} offers this course under the name {2}.\n<ul><li><strong>{3} Video Learning Outcomes</strong> attained across<strong> {4} topics</strong> taught over <strong>{5} hours</strong> of instruction.</li><li>Total duration of course: <strong>{5} hours</strong></li></ul>'

    for univ in university_data:
        d = draft.format(univ.paper, univ.university.upper(), univ.university_subject_name.capitalize(), univ.CO, univ.topics, univ.CO_time)
        final_tabby = final_tabby + d
    
    final_tabby = final_tabby + '[tabbyending]\n\n'
    
    return final_tabby

def generate_description(university_data, university, subject, password, v2l=True, is_diploma=False):    
    button = '''<button class=\"single_add_to_cart_button button alt\" style=\"font-family: \'Open Sans\'; font-size: .8em;\" onclick=\"window.open(\'https://enlp.in/login/index.php\', \'_blank\'); return false;\">View Sample Course</button>\n <strong>Username</strong>: <i>{0}</i>; <strong>Password</strong>: <i>{1}</i>\n\n'''
    if is_diploma:
        top = '''<strong>{1}</strong> paper code(s) are comprehensively covered in <strong>{0}</strong> course.\nCheck out your complete syllabus coverage in <a href="https://cc.2learn.in/det" target="_blank">Curriculum Coverage Plan (CCP)</a>.\n\n'''
    else:
        top = '''<strong>{1}</strong> paper code(s) are comprehensively covered in <strong>{0}</strong> course.\nCheck out your complete syllabus coverage in <a href="https://cc.2learn.in/btech" target="_blank">Curriculum Coverage Plan (CCP)</a>.\n\n'''
    #end = 'Vdo2learn is a shared-library resource requiring no user interaction. Courses are organised by Units, Sections and Topics. Each Topic consists of one or more Video Learning Modules (VLM). Each VLM is taught through several Concept Learning Outcomes (CLO). A CLO is a statement of what a student should expect to recall, solve or perform upon its completion.\n\nCourse Faculty has 10+ years of teaching and industry experience. The lessons are taught systematically, from the very basics, and in simple, lucid language and manner. The Course is designed to inculcate a clear understanding of subject-concepts and enable students to gain theoretical as well as practical insights.\n'
    end = '''V2L courses enjoy a finely granulated hierarchy organised by Units, Sections, Topics, Video Learning Modules (VLM) and Concept Learning Outcomes (CLO).\n\nA CLO is a statement of what a student should expect to recall, solve or perform upon its completion.\n\nCourse Faculty carry several years of teaching and industry experience. The lessons are taught systematically, and delivered in simple, lucid language and manner. Courses are designed to inculcate a clear understanding of subject-concepts and enable students to gain theoretical and practical insights.\n'''
    final_result = button.format(password, password)
    
    name = university.upper() + "-" + subject
    papers = ", ".join([y for x in university_data for y in x.paper.split("/")])
    papers = " and".join(papers.rsplit(",", 1))
    top = top.format(name, papers)
    
    if v2l:
        tabby = get_tabby_data_v2l(university_data)
        top = top + tabby
        
        
    final_result += top
    final_result += end
    return final_result
        

In [22]:
def get_descriptions(csv_folder, out_folder='descriptions_new/', is_diploma=False):
    
    for idx, file in enumerate(os.listdir(csv_folder)):

        csv_file = os.path.join(csv_folder, file)
        df = pd.read_csv(csv_file)
        data = extract_info(df, is_diploma)

        for i, subject in enumerate(data.keys()):
            descriptions = []
            university_data = data[subject]['data']
            password= data[subject]['password']
            for university in university_data.keys():
                description = generate_description(university_data[university], university, subject, password, is_diploma=is_diploma)
                descriptions.append(Description(subject, university, description))

            d = pd.DataFrame([dd.__to_dict__() for dd in descriptions])
            d.to_csv(out_folder + subject + ".csv",index=False)
            if idx==0 and i==0:
                d.to_csv(out_folder + "combined.csv", mode='a', index=False)
            else:
                d.to_csv(out_folder + "combined.csv", mode='a', header=False, index=False)



#### Code for generating Btech CD

In [23]:
csv_dir = "files/v2l-btech"
get_descriptions(csv_dir, 'descriptions_new/btech/')

Engineering Mechanics AKTU 125.0
Engineering Mechanics DBATU 78.0
Engineering Mechanics GGSIPU 68.0
Engineering Mechanics GTU 118.0
Engineering Mechanics JCBUST 79.0
Engineering Mechanics MAKAUT 79.0
Engineering Mechanics MRSPTU 80.0
Engineering Mechanics RTU 83.0
Numerical Methods DBATU 70.0
Numerical Methods DBATU 68.0
Numerical Methods DBATU 78.0
Numerical Methods DBATU 70.0
Numerical Methods GGSIPU 76.0
Numerical Methods GTU 92.0
Numerical Methods GTU 70.0
Numerical Methods MAKAUT 72.0
Numerical Methods MRSPTU 61.0
Engineering Math 1 AKTU 134.0
Engineering Math 1 DBATU 107.0
Engineering Math 1 GGSIPU 122.0
Engineering Math 1 GTU 137.0
Engineering Math 1 IKGPTU 108.0
Engineering Math 1 JCBUST 165.0
Engineering Math 1 JCBUST 246.0
Engineering Math 1 JCBUST 153.0
Engineering Math 1 JCBUST 165.0
Engineering Math 1 MAKAUT 101.0
Engineering Math 1 MAKAUT 165.0
Engineering Math 1 MRSPTU 148.0
Engineering Math 1 RGPV 106.0
Engineering Math 1 RTU 106.0
Engineering Math 2 AKTU 168.0
Engineer

Fluid Mechanics MAKAUT 142.0
Fluid Mechanics MRSPTU 160.0
Fluid Mechanics MRSPTU 132.0
Fluid Mechanics MRSPTU 130.0
Fluid Mechanics MRSPTU 132.0
Fluid Mechanics MRSPTU 135.0
Fluid Mechanics RGPV 142.0
Fluid Mechanics RGPV 122.0
Fluid Mechanics RGPV 142.0
Fluid Mechanics RTU 140.0
Fluid Mechanics RTU 105.0
Fluid Mechanics RTU 159.0
Fluid Mechanics RTU 160.0
Fluid Mechanics RTU 159.0
Strength of Material AKTU 143.0
Strength of Material AKTU 144.0
Strength of Material AKTU 120.0
Strength of Material DBATU 139.0
Strength of Material DBATU 135.0
Strength of Material GGSIPU 128.0
Strength of Material GGSIPU 136.0
Strength of Material GGSIPU 128.0
Strength of Material GTU 132.0
Strength of Material GTU 127.0
Strength of Material IKGPTU 125.0
Strength of Material IKGPTU 125.0
Strength of Material IKGPTU 142.0
Strength of Material IKGPTU 125.0
Strength of Material JCBUST 122.0
Strength of Material JCBUST 139.0
Strength of Material JCBUST 120.0
Strength of Material MAKAUT 146.0
Strength of Mater

#### Code for generating Diploma CD

In [24]:
csv_dir = "files/v2l-diploma"
get_descriptions(csv_dir, 'descriptions_new/diploma/', True)

Applied Math I DLBTE 79
Applied Math I GJBTE 81
Applied Math I HRBTE 125
Applied Math I MHBTE 54
Applied Math I MPBTE 99
Applied Math I PBBTE 99
Applied Math I RJBTE 137
Applied Math I UPBTE 89
Applied Math I WBBTE 103
Applied Math II DLBTE 40
Applied Math II GJBTE 35
Applied Math II MHBTE 56
Applied Math II MPBTE 53
Applied Math II PBBTE 52
Applied Math II UPBTE 51
Applied Math II WBBTE 85
Applied Math III UPBTE 128
RCC DLBTE 69
RCC HRBTE 62
RCC MPBTE 69
RCC PBBTE 67
RCC RJBTE 71
RCC UPBTE 62
RCC WBBTE 69
Soil Mechanics DLBTE 107
Soil Mechanics GJBTE 84
Soil Mechanics HRBTE 107
Soil Mechanics MHBTE 112
Soil Mechanics MPBTE 119
Soil Mechanics PBBTE 107
Soil Mechanics RJBTE 107
Soil Mechanics UPBTE 107
Soil Mechanics WBBTE 115
Surveying DLBTE 52
Surveying DLBTE 37
Surveying GJBTE 72
Surveying GJBTE 43
Surveying HRBTE 59
Surveying HRBTE 56
Surveying MHBTE 57
Surveying MPBTE 61
Surveying MPBTE 48
Surveying PBBTE 59
Surveying PBBTE 56
Surveying RJBTE 61
Surveying RJBTE 43
Surveying UPBTE 5

In [252]:
a = "sdf sdf (sfd)".split("(")[0].strip()
df.loc[df['Subject Name']=='Data Structures', 'S.N'][0]

import os
for file in os.listdir('files'):
    print(file)

List of Subjects B.Tech - CC.csv
List of Subjects B.Tech - CE.csv
List of Subjects B.Tech - CS.csv
List of Subjects B.Tech - EX.csv
List of Subjects B.Tech - ME.csv
